In [6]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

# Load in the card database

In [12]:
df = pd.read_csv('../data/embedding.csv')
df = df.set_index(['Name','Description'])
print(f'length: {len(df)}')
df.sample(2)

length: 2


,
Name,Description
Ancestral Empowerment,"Ancestral Empowerment is a 'Ninja – Attack Reaction' card from the 'Welcome to Rathe' set. It costs 0, pitches for 1, defends for 3, has None power, and has the abilities; Target Ninja attack action card gains +1{p}.\r\n\r\nDraw a card."
Alpha Rampage,"Alpha Rampage is a 'Brute Action – Attack' card from the 'Welcome to Rathe' set. It costs 3, pitches for 1, defends for 3, has 9 power, and has the abilities; **Rhinar Specialization**\r\n\r\nAs an additional cost to play Alpha Rampage, discard a random card.\r\n\r\nWhen you attack with Alpha Rampage, **intimidate**."
